In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import (ImageList, PointsItemList, get_transforms, ResizeMethod, open_image, Tensor, 
                           tensor, Image, imagenet_stats, Flatten, cnn_learner, models, Path)
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
from fastprogress import progress_bar
import numpy as np
import pandas as pd

import torch
from  torch import nn

from henrys_model_utils import MSELossFlat, Reshape

In [3]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [4]:
y = pd.read_csv(labels_path, index_col='filename')

In [5]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [6]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [7]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [8]:
# data.show_batch(3, figsize=(6,6))

In [9]:
# class Reshape(nn.Module):
#     def __init__(self, *args):
#         super(Reshape, self).__init__()
#         self.shape = args

#     def forward(self, x):
#         return x.view(self.shape)
    
# class MSELossFlat(nn.MSELoss):
#     def forward(self, input, target):
#         return super().forward(input.view(-1), target.view(-1))

In [10]:
head_reg = nn.Sequential(
    Flatten(), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(51200, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256, 22),
    Reshape(-1,11,2),
    nn.Tanh()
)

In [11]:
mse_loss_flat = MSELossFlat()

learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=mse_loss_flat,
    custom_head=head_reg
).to_fp16()

In [12]:
learn.path = Path('..')

In [13]:
# learn.lr_find()
# learn.recorder.plot()

In [14]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     10, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=10)
#     ]
# )

In [15]:
learn = learn.load('fastai_resnet34')
learn = learn.to_fp32()

In [16]:
# learn.freeze_to(-2)
# learn.fit_one_cycle(
#     10, 
#     slice(1e-3), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-2'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=10)
#     ]
# )

In [17]:
# learn.show_results(rows=10)

## Deploy to AWS Lambda

In [46]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['fastai', 'henrys_model_utils'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [47]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2019-10-27 15:16:07,247] INFO - Successfully saved Bento 'PetRegression:20191027151536_347C4E' to path: /Users/HenryDashwood/bentoml/repository/PetRegression/20191027151536_347C4E


In [48]:
from bentoml import load

service = load(saved_path)

print(service.predict(data.get(0)))

[2019-10-27 15:16:08,354] WARNING - Module `pet_regression` already loaded, using existing imported module.
(ImagePoints (320, 320), tensor([[-0.2089, -0.2455],
        [-0.4590, -0.1428],
        [-0.4067,  0.1761],
        [-0.3185,  0.4969],
        [-0.0623,  0.4999],
        [ 0.1735,  0.0348],
        [-0.2068, -0.0289],
        [-0.1468,  0.2570],
        [ 0.0906, -0.0429],
        [ 0.0733,  0.0498],
        [ 0.1184,  0.1269]]), tensor([[-0.2089, -0.2455],
        [-0.4590, -0.1428],
        [-0.4067,  0.1761],
        [-0.3185,  0.4969],
        [-0.0623,  0.4999],
        [ 0.1735,  0.0348],
        [-0.2068, -0.0289],
        [-0.1468,  0.2570],
        [ 0.0906, -0.0429],
        [ 0.0733,  0.0498],
        [ 0.1184,  0.1269]]))


In [49]:
!pip install --upgrade {saved_path}

Processing /Users/HenryDashwood/bentoml/repository/PetRegression/20191027151536_347C4E
  Created wheel for PetRegression: filename=PetRegression-20191027151536_347C4E-cp37-none-any.whl size=81246769 sha256=e5a7273bbbb7b40d869566e1523d6978bb40e5b2e44c795fee5d5228958d57ed
  Stored in directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/pip-ephem-wheel-cache-zou6deer/wheels/31/15/c9/17dce9959d16572c86fb4cc1027f02d5bd58ac7aa294053a8b
Successfully built PetRegression
  Found existing installation: PetRegression 20191027150507-1C6DFA
    Uninstalling PetRegression-20191027150507-1C6DFA:
      Successfully uninstalled PetRegression-20191027150507-1C6DFA


In [50]:
!PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

(ImagePoints (320, 320), tensor([[-0.3867,  0.0846],
        [-0.6847,  0.0874],
        [-0.5517,  0.2640],
        [-0.6337,  0.4377],
        [-0.3433,  0.4129],
        [-0.1768,  0.2797],
        [-0.3842,  0.1987],
        [-0.3603,  0.3473],
        [-0.2144,  0.2346],
        [-0.2251,  0.2833],
        [-0.2118,  0.3196]]), tensor([[-0.3867,  0.0846],
        [-0.6847,  0.0874],
        [-0.5517,  0.2640],
        [-0.6337,  0.4377],
        [-0.3433,  0.4129],
        [-0.1768,  0.2797],
        [-0.3842,  0.1987],
        [-0.3603,  0.3473],
        [-0.2144,  0.2346],
        [-0.2251,  0.2833],
        [-0.2118,  0.3196]]))


In [51]:
# !bentoml serve {saved_path}

In [52]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20191027151536_347C4E


In [53]:
!bentoml --verbose deployments delete facial-features-detector --force

[2019-10-27 15:17:25,253] DEBUG - Using BentoML with local Yatai server
[2019-10-27 15:17:25,398] DEBUG - Upgrading tables to the latest revision
Successfully deleted deployment "facial-features-detector"


In [54]:
!bentoml --verbose deployment create facial-features-detector --bento {bento_tag} --platform aws-lambda --region us-east-1

[2019-10-27 15:17:28,669] DEBUG - Using BentoML with local Yatai server
[2019-10-27 15:17:28,799] DEBUG - Upgrading tables to the latest revision
[2019-10-27 15:17:29,605] DEBUG - Created temporary directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-xa4ebrj1
[2019-10-27 15:18:14,281] DEBUG - sls cmd output: Serverless: Generating boilerplate...
 _______                             __
|   _   .-----.----.--.--.-----.----|  .-----.-----.-----.
|   |___|  -__|   _|  |  |  -__|   _|  |  -__|__ --|__ --|
|____   |_____|__|  \___/|_____|__| |__|_____|_____|_____|
|   |   |             The Serverless Application Framework
|       |                           serverless.com, v1.53.0
 -------'

Serverless: Successfully generated boilerplate for template: "aws-python3"

[2019-10-27 15:18:18,079] INFO - Installing additional packages: serverless-python-requirements
[2019-10-27 15:18:26,824] DEBUG - sls cmd output: Serverless: Creating an empty package.json file in your

In [28]:
# !bentoml deployment list